In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import cv2
from torchvision.transforms import Normalize

In [2]:
video = cv2.VideoCapture('nine_hour_video\Video.mp4')
c = 0
while True:
    c += 1
    ret, frame = video.read()
    cv2.imshow('part', frame[:, frame.shape[1] // 2:])
    if cv2.waitKey(1) & 0xFF == ord('q') or c == 60 * 25:
       break

video.release()
cv2.destroyAllWindows()

In [3]:
def get_right_left(left_pd, right_pd, vid_len):
    dict_type = {'простой': 0, 'вынужденная': 1, 'сварка': 2, 'перерыв': 3, 'обед': 3}
    right_pd['sec'] = right_pd['time'].apply(lambda x: (int(x.split(':')[0]) * 3600 + int(x.split(':')[1]) * 60))
    right_pd['new_type'] = right_pd['type'].apply(lambda x: dict_type[x])
    left_pd['sec'] = left_pd['time'].apply(lambda x: (int(x.split(':')[0]) * 3600 + int(x.split(':')[1]) * 60))
    left_pd['new_type'] = left_pd['type'].apply(lambda x: dict_type[x])
    left = np.zeros(60 * 60 * vid_len)
    right = np.zeros(60 * 60 * vid_len)
    for second, typ in left_pd[['sec', 'new_type']].to_numpy():
        left[second:] = typ
    for second, typ in right_pd[['sec', 'new_type']].to_numpy():
        right[second:] = typ
    return left, right


In [4]:
def get_valid_by_min(path, mini, right_y, left_y):
        video = cv2.VideoCapture(path)
        fps = video.get(cv2.CAP_PROP_FPS)
        frame_skip = int(fps) * 60
        c = 0
        data_X = []
        data_y = []
        current_frame = 0
        while True:
            ret, frame = video.read()
            left_img = frame[:, :frame.shape[1] // 2]
            right_img = frame[:, frame.shape[1] // 2:]
            left_img = np.array(cv2.split(left_img))
            right_img = np.array(cv2.split(right_img))
            data_X.append(left_img.astype('float64')) if left_y[c * 60] != 3 else 0
            data_y.append(left_y[c * 60]) if left_y[c * 60] != 3 else 0
            data_X.append(right_img.astype('float64')) if right_y[c * 60] != 3 else 0
            data_y.append(right_y[c * 60]) if right_y[c * 60] != 3 else 0
            current_frame += frame_skip
            video.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
            if c == mini - 1:
                break
            c += 1
        video.release()
        return np.array(data_X), np.array(data_y)
        

In [6]:
full_5hours = get_valid_by_min('five_hour_video\Video.mp4', 300, *get_right_left(pd.read_csv(r'csv\LabelsLeft_five_hour_video.csv'), pd.read_csv('csv\LabelsRight_five_hour_video.csv'), 5))

In [ ]:
#full_9hours = get_valid_by_min('five_hour_video\Video.mp4', 540, *get_right_left(pd.read_csv(r'csv\LabelsLeft_nine_hour_video.csv'), pd.read_csv('csv\LabelsRight_nine_hour_video.csv'), 9))

In [7]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def train_model(model, optimizer, loss_function, data):    
    model.train()
    all_loss = 0
    X_all, y_all = data[0], data[1]
    y_pred = []
    step = 5
    for it in range(0, len(X_all), step):
        X = torch.from_numpy(X_all[it: it + step]).to(torch.float32)
        y = torch.from_numpy(y_all[it: it + step]).to(torch.long)
        pred = model(X)
        y_pred.extend(list(pred))
        loss = loss_function(pred, y)
        print(f"Current loss = {loss.item()}")
        all_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    y_pred = [list(x).index(max(x)) for x in y_pred]
    print(f"Accuracy: {accuracy_score(y_all, y_pred)}")
    print(f"Presicion: {precision_score(y_all, y_pred, average='weighted')}")
    print(f"Recall: {recall_score(y_all, y_pred, average='weighted')}")
    print(f"F1 score: {f1_score(y_all, y_pred, average='weighted')}")
    print(f'Потеря на тестовой выборке {all_loss / (len(X_all) // step)}')
    return all_loss / (len(X_all) // step)


def test_model(model, loss_function, data):    
    model.eval()
    all_loss = 0
    X_all, y_all = data[0], data[1]
    y_pred = []
    step = 5
    for it in range(0, len(X_all), step):
        X = torch.from_numpy(X_all[it: it + step]).to(torch.float32)
        y = torch.from_numpy(y_all[it: it + step]).to(torch.long)
        pred = model(X)
        y_pred.extend(list(pred))
        loss = loss_function(pred, y)
        all_loss += loss.item()
        loss.backward()
    y_pred = [list(x).index(max(x)) for x in y_pred]
    print(f"Accuracy: {accuracy_score(y_all, y_pred)}")
    print(f"Presicion: {precision_score(y_all, y_pred, average='weighted')}")
    print(f"Recall: {recall_score(y_all, y_pred, average='weighted')}")
    print(f"F1 score: {f1_score(y_all, y_pred, average='weighted')}")
    print(f'Потеря на тестовой выборке {all_loss / (len(X_all) // step)}')
    return all_loss / (len(X_all) // step)

In [15]:
model = nn.Sequential(
    nn.Conv2d(3, 4, 3),
    nn.ReLU(),
    nn.MaxPool2d(3, 3),

    nn.Conv2d(4, 16, 3),
    nn.ReLU(),
    nn.MaxPool2d(3, 3),

    nn.Conv2d(16, 64, 3),
    nn.ReLU(),
    nn.MaxPool2d(3, 3),

    nn.Conv2d(64, 128, 3),
    nn.ReLU(),
    nn.MaxPool2d(4, 4),

    nn.Flatten(-3),

    nn.BatchNorm1d(9216),
    nn.Linear(9216, 3200),
    nn.ReLU(),
    nn.Linear(3200, 320),
    nn.ReLU(),
    nn.Linear(320, 3),
    nn.Softmax()
)
optim = torch.optim.Adam(model.parameters(), lr=1e-1)
loss = nn.CrossEntropyLoss()

In [16]:
train_model(model, optim, loss, full_5hours)

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Current loss = 1.110744595527649
Current loss = 1.151444673538208
Current loss = 1.55144464969635
Current loss = 0.9514446258544922
Current loss = 0.5514446496963501
Current loss = 0.7514446377754211
Current loss = 1.55144464969635
Current loss = 1.55144464969635
Current loss = 1.55144464969635
Current loss = 1.151444673538208
Current loss = 1.55144464969635
Current loss = 1.151444673538208
Current loss = 1.151444673538208
Current loss = 1.151444673538208
Current loss = 1.151444673538208
Current loss = 1.151444673538208
Current loss = 1.3514446020126343
Current loss = 1.55144464969635
Current loss = 1.55144464969635
Current loss = 0.9514446258544922
Current loss = 1.55144464969635
Current loss = 1.55144464969635
Current loss = 1.55144464969635
Current loss = 1.55144464969635
Current loss = 1.55144464969635
Current loss = 1.55144464969635
Current loss = 1.55144464969635
Current loss = 1.55144464969635
Current loss = 1.55144464969635
Current loss = 1.55144464969635
Current loss = 1.55144

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1.2529196435654606

In [ ]:
test_model(model, loss, get_valid_by_min('nine_hour_video\Video.mp4', 120, *get_right_left(pd.read_csv('csv\LabelsLeft_nine_hour_video.csv'), pd.read_csv('csv\LabelsRight_nine_hour_video.csv'), 9)))

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Accuracy: 0.24583333333333332
Presicion: 0.21864583333333332
Recall: 0.24583333333333332
F1 score: 0.2239203276247208
Потеря на тестовой выборке 1.1737496107816696


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1.1737496107816696

In [ ]:
new_model = nn.Sequential(
    nn.BatchNorm2d(3),
    nn.Conv2d(3, 4, 3),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Conv2d(4, 16, 3),
    nn.ReLU(),
    nn.MaxPool2d(3, 3),

    nn.Conv2d(16, 64, 3),
    nn.ReLU(),
    nn.MaxPool2d(4, 4),

    nn.Conv2d(64, 128, 3),
    nn.ReLU(),
    nn.MaxPool2d(5, 5),


    nn.Flatten(-3),

    nn.BatchNorm1d(7168),
    nn.Linear(7168, 3200),
    nn.ReLU(),
    nn.Linear(3200, 320),
    nn.ReLU(),
    nn.Linear(320, 3),
    nn.Softmax()
)
optim_new = torch.optim.Adam(new_model.parameters(), lr=1e-1)
loss_new = nn.CrossEntropyLoss() 

In [ ]:
train_model(new_model, optim_new, loss_new, full_5hours)

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Current loss = 1.1128147840499878
Current loss = 1.2514448165893555
Current loss = 0.6514445543289185
Current loss = 1.5514447689056396
Current loss = 1.3514448404312134
Current loss = 1.3514448404312134
Current loss = 1.251444697380066
Current loss = 1.251444697380066
Current loss = 1.4514448642730713
Current loss = 1.251444697380066
Current loss = 1.5514447689056396
Current loss = 1.5514447689056396
Current loss = 1.5514447689056396
Current loss = 1.5514447689056396
Current loss = 1.5514447689056396
Current loss = 1.5514447689056396
Current loss = 1.4514448642730713
Current loss = 0.7514445781707764
Current loss = 0.7514445781707764
Current loss = 0.8514446020126343
Current loss = 1.3514448404312134
Current loss = 1.251444697380066
Current loss = 1.4514448642730713
Current loss = 1.151444673538208
Current loss = 1.3514448404312134
Current loss = 0.9514446258544922
Current loss = 0.6514445543289185
Current loss = 1.0514447689056396
Current loss = 0.7514445781707764
Current loss = 0.95

1.2544330755869548

In [ ]:
test_model(new_model, loss_new, get_valid_by_min('nine_hour_video\Video.mp4', 120, *get_right_left(pd.read_csv('csv\LabelsLeft_nine_hour_video.csv'), pd.read_csv('csv\LabelsRight_nine_hour_video.csv'), 9)))

In [17]:
import os
save_folder = 'model'

save_path = os.path.join(save_folder, 'new_model.pth')
save_path2 = os.path.join(save_folder, 'our_model.pth')


#torch.save(new_model, save_path)
torch.save(model, save_path2)